<div class="markdown-google-sans">

# <strong>Bosch Assessment</strong>

## <strong>Overview</strong>
</div>

The objective is to perform a thorough analysis of this data and build a machine learning model to predict a target variable (price).

<div class="markdown-google-sans">
  <h3>Packages Installation</h3>
</div>

In [20]:
%pip install ucimlrepo
%pip install pandas
%pip install matplotlib

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
Note: you may need to restart the kernel to use updated packages.
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Home

<div class="markdown-google-sans">
  <h3>Imports</h3>
</div>

In [21]:
import pandas as pd

<div class="markdown-google-sans">

## <strong>Exploratory Data Analysis (EDA)</strong>
</div>

Conduct a thorough exploratory data analysis. 

This should include understanding the distribution of data, detecting outliers, and exploring relationships between features. 

Visualize important features and correlations.

<div class="markdown-google-sans">

## <strong>Feature Engineering and Selection</strong>
</div>

Based on your EDA, engineer new features and select the most relevant ones for your model. Justify your choices.

<div class="markdown-google-sans">

## <strong>Machine Learning Model</strong>
</div>

Build a machine learning model to predict the "price" variable.
Explain your choice of model and any hyperparameters you tune. Use appropriate validation techniques.

<div class="markdown-google-sans">

## <strong>Evaluation and Interpretation</strong>
</div>

Evaluate the performance of your model using appropriate metrics. 
Interpret your model's predictions, and discuss its strengths and weaknesses.